In [37]:
import pandas as pd; import seaborn as sb; import statsmodels.formula.api as smf; import pingouin;
from itertools import combinations; import os; from analysis_helpers import *; 
import scipy; from scipy import stats; import os; import math; import warnings
warnings.filterwarnings('ignore')

# Check data files

This code checks the number of participants in each experiment and verifies the number of each participant's files (and file sizes)

In [29]:
def gaze_file_check(exps, grps, full=False):
    '''
    checks the number and size of gaze files for each subject in each experiment and group passed
    does not return anything, prints results
    '''
    for exp in exps:  #['sustained_attention_experiment', 'variable_attention_experiment']:
        for grp in grps:  #['group1', 'group2']:

            print(); print('##################################'); print()
            combo = '../'+exp+'/data/'+grp+'/'
            print(str(combo)); print()
            print(str(len([g for g in os.listdir(combo) if g != '.DS_Store'])) + ' participants'); print()
            # print('##################################'); print()

            missing_gaze = 0; funny_size = 0
            
            for idx,x in enumerate([g for g in os.listdir(combo) if g != '.DS_Store']):

                loop = 0; 
                gaze_files = [a for a in os.listdir(combo+x+'/eye_data') if a != '.DS_Store']

                if len(gaze_files) <8:
                    if full == True:
                        print(x); print(); print( '*** '+ str(len(gaze_files)) + ' gaze files ***'); print(); 
                    loop += 1
                    missing_gaze += 1

                file_sizes = [os.path.getsize(combo+x+'/eye_data'+'/'+k) for k in gaze_files]
                indices = [i for i, j in enumerate(file_sizes) if j < 2000000 or j > 10000000] 

                if len(indices) > 0:
                    if full == True:
                        print(x); print()

                    for ind in indices:
                        if full == True:
                            print(gaze_files[ind]); print(file_sizes[ind]); print(); 
                        loop += 1
                    funny_size += len(indices)

                if loop >0 and full == True: 
                    print('----------------- ')

            if full == False:
                print(str(missing_gaze)+' missing gaze files')
                print(str(funny_size)+' large or small gaze files')


In [30]:
gaze_file_check(['sustained_attention_experiment', 'variable_attention_experiment'], ['group1', 'group2'])


##################################

../sustained_attention_experiment/data/group1/

30 participants

1 missing gaze files
7 large or small gaze files

##################################

../sustained_attention_experiment/data/group2/

30 participants

0 missing gaze files
34 large or small gaze files

##################################

../variable_attention_experiment/data/group1/

30 participants

3 missing gaze files
15 large or small gaze files

##################################

../variable_attention_experiment/data/group2/

23 participants

0 missing gaze files
46 large or small gaze files


# Check parsed gaze data

How many gazepoints are recorded during each presentation trial (3s) on average?
Are there trials where none are captured? Are there duplicate gaze data points?

In [39]:
gaze       = pd.read_csv('../parsed_data/full_gaze.csv')
behavioral = pd.read_csv('../parsed_data/full_behavioral.csv')

for s in behavioral['UniqueID'].unique():
    for r in behavioral['Run'].unique():
        behavioral.loc[(behavioral['Run']==r) 
                 & (behavioral['UniqueID']==s) 
                 & (behavioral['Trial Type']=='Memory'), 'Last Cued'] = behavioral[(behavioral['Run']==r) & (behavioral['UniqueID']==s) & (behavioral['Trial Type']=='Presentation') & (behavioral['Trial']==9)]['Cued Category'].item()

In [40]:
os.listdir('../parsed_data')

['.DS_Store', 'full_behavioral.csv', 'README.md', 'full_gaze.csv']

# Check oddball gaze files

Do gaze files that are unusually large or small catch more or less gazepoints during viewing?